In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [ ]:
!pip install transformers datasets accelerate peft trl bitsandbytes

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "vivivivivivi/Llama-2-7b-chat-finetune"

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}


In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline
prompt=""
query = "How is a 3D model uploaded into the 3D Viewer app on ctrlX CORE?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
context= " "

In [ ]:
result = pipe(f"""You are an AI assistant with access to a vast external knowledge base. Given the following context and query, generate a comprehensive answer based on the query. Don't hallucinate and restrict yourself with the following context.

### Context:
{context}

### Query:
{query}

Use the retrieved information to craft a detailed, accurate, and coherent response.""")

In [ ]:
print(result[0]['generated_text'])

You are an AI assistant with access to a vast external knowledge base. Given the following context and query, generate a comprehensive answer based on the query. Don't hallucinate and restrict yourself with the following context.

### Context:
 

### Query:
How is a 3D model uploaded into the 3D Viewer app on ctrlX CORE?

Use the retrieved information to craft a detailed, accurate, and coherent response. Please provide a step-by-step guide that describes the entire process of uploading a 3D model into the 3D Viewer app on ctrlX CORE. Your answer should be clear, concise, and easy to follow. If possible, please provide screenshots or links to additional resources for further clarification.
